Статьи для чтения
*   [~~Частотный анализ~~](https://habr.com/ru/articles/583304/)
*   [Необоснованная эффективность рекуррентных нейронных сетей](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)
*   [Fun with RNNs](https://www.cs.toronto.edu/~guerzhoy/321/proj4/)
*   http://shannon.usu.edu.ru/Shannon/shannon1948/node2.html
*   [Рекуррентные нейронные сети (RNN) с Keras](https://habr.com/ru/articles/487808/)
* https://www.tensorflow.org/text/tutorials/text_generation?hl=ru
* https://skine.ru/articles/734433/



##Частотный анализ

In [1]:
%pip install frequency-analysis
%pip install beautifulsoup4 lxml
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import io
import os
import string
from datetime import datetime
from os import listdir
import frequency_analysis
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def clear_and_remove_dir(dir):
    if os.path.exists(dir + '/.ipynb_checkpoints'):
        os.rmdir(dir + '/.ipynb_checkpoints')
    if os.path.exists(dir):
        remove_files(dir)
        os.rmdir(dir)

In [4]:
def remove_files(dir):
    files = listdir(dir)
    for n, file in enumerate(files):
        os.remove(dir + '/' + file)

In [5]:
def build_xls_report(dir, yo_enabled):
    with frequency_analysis.Result(name = dir) as res:
        res.treat(limits=(1000,) * 4, chart_limits=(20,) * 4, min_quantities=(10,) * 5)
        res.sheet_ru_top_symbols()  # optional argument – chart_limit (int)
        ru_symbs = 'абвгдежзийклмнопрстуфхцчшщьыъэюяz!?.,'
        res.sheet_custom_symbol_bigrams(ru_symbs, ignore_case=True, name='Russian letter bigrams')
        if yo_enabled:
            res.sheet_yo_words()

In [6]:
def tokinize(text):
    return word_tokenize(text)

In [7]:
def clear_text(text):
    spec_chars = '\n\xa0«»\t—…{}[]()' 
    cleared = text.lower()
    cleared = remove_chars_from_text(cleared, spec_chars)
    cleared = remove_chars_from_text(cleared, string.digits)
    return cleared

In [8]:
def print_progress_to_file(n, length, name):
    with open('progress.txt', 'a') as f:
        print(n, "/", length, name, datetime.now().strftime('%H:%M:%S'), file=f)

In [9]:
def split_to_sentence(text):
    return text.split(".")

In [10]:
def remove_chars_from_text(text, chars):
    return "".join([ch for ch in text if ch not in chars])

In [11]:
def xml_analysis(input_dir, output_dir, yo_enabled):
    clear_and_remove_dir(output_dir)
    file_list = listdir(input_dir)
    start = datetime.now()
    with frequency_analysis.Analysis(name = output_dir, yo=yo_enabled) as analyze:
        for n, file in enumerate(file_list):
            with io.open(input_dir + '/' + file, mode='r', encoding='utf-8') as f:
                data = f.read()
            bs_data = BeautifulSoup(input_dir, 'xml')
            for sentence in bs_data.find_all('source'):
                printsentence.text.split()
                analyze.count_symbols(sentence.text.split(), pos=True)
            print(n, file)
    print('fin at:', datetime.now().strftime('%H:%M:%S'))
    print('total time taked to analysis:', datetime.now() - start)
    build_xls_report(output_dir, yo_enabled)

In [12]:
def txt_analysis(input_dir, output_dir, yo_enabled):
    clear_and_remove_dir(output_dir)
    file_list = listdir(input_dir)
    word_pattern = '[a-zA-Zа-яА-ЯёЁ]+(?:(?:-?[a-zA-Zа-яА-ЯёЁ]+)+|\'?[a-zA-Zа-яА-ЯёЁ]+)|[a-zA-Zа-яА-ЯёЁ]';
    start = datetime.now()
    with frequency_analysis.Analysis(name = output_dir, yo=yo_enabled, word_pattern=word_pattern) as analyze:
        for n, file in enumerate(file_list):
            print_progress_to_file(n, len(file_list), file)
            with io.open(input_dir + '/' + file, mode='r', encoding='utf-8') as f:
                try:
                    text = f.read()
                except Exception:
                    print('Can not read file', f)
                    continue
            sentences = sent_tokenize(text)
            for sentence in sentences:
                if len(sentence) > 1:
                    buffer = []
                    for word in sentence.split():
                        word = word + "z"
                        buffer.append(word)
                    analyze.count_symbols(buffer, pos=True)
    print('fin at:', datetime.now().strftime('%H:%M:%S'))
    print('total time taked to analysis:', datetime.now() - start)
    build_xls_report(output_dir, yo_enabled)

In [13]:
def my_analysis(input_dir, output_dir, input_text_format, yo_enabled=False):
    if(input_text_format == 'txt'):
        txt_analysis(input_dir, output_dir, yo_enabled)
    else:
        xml_analysis(input_dir, output_dir, yo_enabled)
    

##Анализ произведений Пушкина

In [14]:
my_analysis('test3', 'test3_analysis', 'txt', False)

fin at: 14:58:24
total time taked to analysis: 0:00:08.810890
Start of writing to .xlsx
... "Stats" sheet was written
... "Top symbols" sheet was written
... "Top symbol bigrams" sheet was written
... "All symbol bigrams" table sheet was written
... "Top words" sheet was written
... "Top word bigrams" sheet was written
End of writing main sheets.
You can call additional functions to create more sheets (e.g. "sheet_en_symbol_bigrams()", "sheet_ru_symbol_bigrams()", "sheet_en_top_symbols([chart_limit])", "sheet_ru_top_symbols([chart_limit]), ""sheet_yo_words([limit, min_quantity])"), "sheet_custom_top_symbols(symbols_str)" or "sheet_custom_symbol_bigrams(symbols_str)".
You can also call 2D sheet functions with "ignore_case=True" argument.
... "Russian letters top" sheet was written.
... "Russian letter bigrams (I)" sheet was written.


In [ ]:
%%time


# remove_files('Lexica' + '/.ipynb_checkpoints')
# os.rmdir('Lexica' + '/.ipynb_checkpoints')

my_analysis("Lexica", "lexica_analysis", "txt", False)

In [29]:
build_xls_report("lexica_analysis", False)

Start of writing to .xlsx
... "Stats" sheet was written
... "Top symbols" sheet was written
... "Top symbol bigrams" sheet was written
... "All symbol bigrams" table sheet was written
... "Top words" sheet was written
... "Top word bigrams" sheet was written
End of writing main sheets.
You can call additional functions to create more sheets (e.g. "sheet_en_symbol_bigrams()", "sheet_ru_symbol_bigrams()", "sheet_en_top_symbols([chart_limit])", "sheet_ru_top_symbols([chart_limit]), ""sheet_yo_words([limit, min_quantity])"), "sheet_custom_top_symbols(symbols_str)" or "sheet_custom_symbol_bigrams(symbols_str)".
You can also call 2D sheet functions with "ignore_case=True" argument.
... "Russian letters top" sheet was written.
... "Russian letter bigrams (I)" sheet was written.
